# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [ ]:
# load in the data
azdias = pd.read_csv('./Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv('./Udacity_CUSTOMERS_052018.csv', sep=';')


/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# Be sure to add in a lot more cells (both markdown and code) to document your
# approach and findings!
#azdias['CAMEO_INTL_2015']

### Data Cleaning
#### 1. Data Cleaning - convert unknown values to `NaN`
The first step is to look at the features and the range of possible of each range. The description of each column can be found in `DIAS Information Levels - Attributes 2017.xlsx` and `DIAS Attributes - Values 2017.xlsx`. The values of each feature contains different ranges, and some represents "unknown". The first step is to manually convert these "unknowns" to NaN.


##### a) read features from attribute file

In [ ]:
ATTRIBUTE_FILE="DIAS Attributes - Values 2017.xlsx"

df_Levels_Attributes = pd.read_excel(ATTRIBUTE_FILE)
#print(df_Levels_Attributes.head())
for i in range(len(df_Levels_Attributes["Attribute"])):
    if pd.isna(df_Levels_Attributes.iloc[i]["Attribute"]):
        df_Levels_Attributes.at[i,"Attribute"] = attr_name
    else:
        attr_name = df_Levels_Attributes.iloc[i]["Attribute"]

COLUMN_UNKNOWN_VALUE = dict()
for index, row in df_Levels_Attributes[df_Levels_Attributes["Meaning"] == "unknown"].iterrows():
    COLUMN_UNKNOWN_VALUE[row["Attribute"]] = [int(n) for n in str(row["Value"]).split(",")]
#print(sorted([key for key in azdias.columns]))

INTEGER_LABELS_COLUMN = {"ANZ_HAUSHALTE_AKTIV", "ANZ_HH_TITEL", "ANZ_PERSONEN", 
                         "ANZ_TITEL", "GEBURTSJAHR", "KBA13_ANZAHL_PKW", "MIN_GEBAEUDEJAHR"}
COLUMN_LABELS = dict()
for index, row in df_Levels_Attributes[df_Levels_Attributes["Meaning"] != "unknown"].iterrows():
    if row["Attribute"] not in COLUMN_LABELS:
        COLUMN_LABELS[row["Attribute"]] = []
    if row["Attribute"] in INTEGER_LABELS_COLUMN:
        continue
    #print(row["Attribute"], row["Value"])
    COLUMN_LABELS[row["Attribute"]] += str(row["Value"]).split(",")
#COLUMN_LABELS["CAMEO_DEU_2015"]
#azdias[azdias["CAMEO_DEU_2015"] == "XX"]["CAMEO_DEU_2015"]
#azdias.loc[2511,"CAMEO_DEU_2015"] in COLUMN_LABELS["CAMEO_DEU_2015"]
#COLUMN_UNKNOWN_VALUE["CAMEO_DEU_2015"]
#set(COLUMN_LABELS.keys()) - set(azdias.columns)
#(set(azdias.columns) - set(COLUMN_LABELS.keys()))
print(set(customers.columns) - set(azdias.columns))

##### b) Drop undocumented features
Many features in the azdias and customers dataframe are not explained in the attribute file. These columns are dropped.

In [ ]:
UNDOCUMENTED_COLUMN = set(azdias.columns) - set(COLUMN_LABELS.keys())
print(UNDOCUMENTED_COLUMN)

azdias = azdias.drop(columns={"LNR"})
customers = customers.drop(columns="LNR")
#print(customers["ONLINE_PURCHASE"])

##### c) Convert values that represents "unknown" to `NaN`
In some columns, the value of `0` or `-1` actually means unknown,or some values are out-of-range. These data are replaced with `NaN`.

In [ ]:
def fill_unknown_values(df):
    MISSING_COLUMN = set(COLUMN_LABELS.keys()) - set(df.columns)
    for column_name in COLUMN_LABELS:
        if column_name in MISSING_COLUMN:
            continue
        if column_name in INTEGER_LABELS_COLUMN:
            continue
        nan_mask = ~df[column_name].isin(COLUMN_LABELS[column_name])
        if column_name in COLUMN_UNKNOWN_VALUE:
            nan_mask |= df[column_name].isin(COLUMN_UNKNOWN_VALUE[column_name])
        df.loc[nan_mask, column_name] = np.nan

    # special handling
    df.loc[df["GEBURTSJAHR"] == 0, "GEBURTSJAHR"] = np.nan

#print(azdias.head())    
fill_unknown_values(azdias)
#print(azdias.head())

#print(customers.head)    
fill_unknown_values(customers)
#print(customers.head)

##### d) Handle numerical strings

In [ ]:
def convert_str_to_int(df, column):
    df[column] = pd.to_numeric(df[column], errors='coerce')

convert_str_to_int(azdias, "CAMEO_DEUG_2015")
convert_str_to_int(azdias, "CAMEO_INTL_2015")
convert_str_to_int(customers, "CAMEO_DEUG_2015")
convert_str_to_int(customers, "CAMEO_INTL_2015")


azdias['EINGEFUEGT_AM'] = pd.to_datetime(azdias['EINGEFUEGT_AM']).sub(pd.Timestamp('1900-01-01')).dt.days
customers['EINGEFUEGT_AM'] = pd.to_datetime(customers['EINGEFUEGT_AM']).sub(pd.Timestamp('1900-01-01')).dt.days

##### e) Check the percetage of NaN values in each column
The majority of NaN is less than 0.15% and at most 0.40% of total features in both of dataset, so it would not have significant impact on the results. 

In [ ]:
azdias_nan_counter = {column_name: np.sum(azdias[column_name].isna()) for column_name in azdias.columns}
customers_nan_counter = {column_name: np.sum(customers[column_name].isna()) for column_name in customers.columns}
#print([c for c in customers_nan_counter if customers_nan_counter[c] > 0])
azdias_na_percent = [100.0*v/float(azdias.size) for v in azdias_nan_counter.values()]
customers_na_percent = [100.0*v/float(customers.size) for v in customers_nan_counter.values()]

plt.hist([azdias_na_percent, customers_na_percent], bins = 25, label=["azdias nan", "customers nan"])

# Plot formatting
plt.legend()
plt.xlabel('Percentage of NaN in column (%)')
plt.ylabel('Number of columns')
plt.title('Histogram')

#### Data Cleaning - Handling catagorized values

In [ ]:
def convert_biclass_to_int(df, column):
    nan_mask = df[column].notna()
    df.loc[nan_mask, column] = df[column][nan_mask].astype('category').cat.codes

convert_biclass_to_int(azdias, "OST_WEST_KZ")
convert_biclass_to_int(customers, "OST_WEST_KZ")
print(azdias.groupby("OST_WEST_KZ").groups.keys())
print(customers.groupby("OST_WEST_KZ").groups.keys())

In [ ]:
def convert_categories_to_onehot(df, column):
    nan_mask = df[column].notna()
    df.loc[nan_mask, column] = df[column][nan_mask].astype('category')
    onehot = pd.get_dummies(df[column], prefix=column, dummy_na=True)
    #print(df.columns)
    df = pd.concat([df, onehot], axis=1)
    df = df.drop(columns=[column])
    #print(df.columns)
    return df
#pd.get_dummies(azdias["CAMEO_DEU_2015"], prefix="CAMEO_DEU_2015", dummy_na=True)
azdias = convert_categories_to_onehot(azdias, "CAMEO_DEU_2015")
customers = convert_categories_to_onehot(customers, "CAMEO_DEU_2015")
azdias = convert_categories_to_onehot(azdias, "D19_LETZTER_KAUF_BRANCHE")
customers = convert_categories_to_onehot(customers, "D19_LETZTER_KAUF_BRANCHE")

In [ ]:

def print_groups(df):
    for column_name in df.columns:
        keys = df.groupby(column_name).groups.keys()
        if set(keys) <= set(range(25)):
            continue
        print("***** {} *****".format(column_name))
        print(keys)


print_groups(azdias)
print_groups(customers)
#customers.groupby("ONLINE_PURCHASE").groups.keys()

#### Data Cleaning - Use mean instead of `Nan`

In [ ]:
columns = list(customers.columns)
columns.remove("CUSTOMER_GROUP")
columns.remove("ONLINE_PURCHASE")
columns.remove("PRODUCT_GROUP")

for c in columns:
    customers[c].fillna(customers[c].mean(), inplace=True)
    azdias[c].fillna(azdias[c].mean(), inplace=True)
print([col for col in customers.columns if np.sum(customers[col].isna()) > 0])
print([col for col in azdias.columns if np.sum(azdias[col].isna()) > 0])


In [ ]:
azdias.describe()

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

### 1. Normalize data

In [ ]:
#mean_azdias = azdias.mean()
#std_azdias = azdias.std()
#normalized_azdias = (azdias - mean_azdias)/std_azdias
max_azdias = azdias.max()
min_azdias = azdias.min()
normalized_azdias = (azdias - min_azdias)/(max_azdias - min_azdias)
normalized_azdias.describe()


### 2. PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
azdias_rotated = pca.fit_transform(normalized_azdias)
explained_variance_by_k = pca.explained_variance_ratio_.cumsum()
plt.plot(range(1,len(explained_variance_by_k)+1),explained_variance_by_k,marker="*")
plt.xlabel("Num. of Components")
plt.ylabel("Cumulative Power Ratio")

The first 150 componants accommodate 90% of energy. However, there is no significant transition in the energy. So we can use the entire feature sets for now.

### 3. K-Means Clustering

#### The elbow is around 260 clusters.

In [ ]:
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import silhouette_score

# choose best K (i.e., number of clusters)
inertias = []
silhouettes = []

print(normalized_azdias.shape)

inertias_mini = []
silhouettes_mini = []
ks = range(20,401,10)
for k in ks:
    print("**** {} ****".format(k))
    kmeans = MiniBatchKMeans(n_clusters=k, verbose=0)
    kmeans.fit(normalized_azdias.values)
    #print("**** 2 ****")
    inertias_mini.append(kmeans.inertia_)
    #print("**** 3 ****")
    #silhouettes_mini.append(silhouette_score(azdias.values, kmeans.predict(azdias.values)))
    #print("**** 4 ****")

In [ ]:
fig = plt.figure(figsize=(20,5))

fig.add_subplot(1,2,1)
plt.plot(ks,inertias_mini,marker='x')# want to use elbow method to find best k

#fig.add_subplot(1,2,2)
#plt.plot(ks,silhouettes_mini,marker='o')# the higher the better
plt.xlabel("Num. of Cluster")
plt.ylabel("Inertia")

#### The elbow is around 260, the kmeans is re-trained and predicts.

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=260, verbose=1)
kmeans.fit(normalized_azdias.values)

In [ ]:
normalized_customers = (customers[azdias.columns] - min_azdias)/(max_azdias - min_azdias)
print([k for k in normalized_customers.columns if np.sum(normalized_customers[k].isna()) > 0])
pred = kmeans.predict(normalized_customers[list(azdias.columns)].values)
pred = pd.DataFrame({'cluster': pred})
print(pred)
#customers_pred = pd.concat([customers, pred])
#print(set(customers.columns) - set(azdias.columns))
#print(customers.groupby('CUSTOMER_GROUP').groups.keys())
#print(customers.groupby('PRODUCT_GROUP').groups.keys())
#print(customers.groupby('ONLINE_PURCHASE').groups.keys())
cluster_group = pred.groupby('cluster').groups
print(cluster_group.keys())

#### Based on the clustering results, the top 1 cluster is cluster-51, which accounts for 12.25% of entire customer base.

In [ ]:
import seaborn as sns
print(pred.shape)
sorted_cluster = sorted([(len(cluster_group[k]), k, len(cluster_group[k])/float(pred.shape[0])) for k in cluster_group], reverse=True)
print(sorted_cluster[:10])
print("\n".join(["{}\t{}\t{:.3f}".format(v[1],v[0],100.0*v[2]) for v in sorted_cluster[:10]]))
print(sum([v[2] for v in sorted_cluster[:10]]))
#print(set(list(kmeans.labels_)))
centroid_id = sorted_cluster[0][1]
print(centroid_id)

#print(mean_azdias)
#print(std_azdias)

def display_component(centroid, max_azdias, min_azdias, n_weights = 25):
    
    #centroid = (centroid + mean_azdias)*std_azdias
    #print(azdias)
    comp = pd.DataFrame(list(zip(centroid, list(azdias.columns))), columns=['scaled_weights', 'features'])
    comp['abs_weights'] = comp['scaled_weights'].apply(lambda x: np.abs(x))
    #print(comp)
    comp['original_weights'] = comp.apply(lambda row: (row.scaled_weights*(max_azdias[row.features] - min_azdias[row.features]) + min_azdias[row.features]), axis = 1) 
    sorted_weight_data = comp.sort_values('abs_weights', ascending=False).head(n_weights)
    #print(sorted_weight_data)
    fig = plt.figure(figsize=(15,20))
    sns.set(style="whitegrid")
#plt.plot(ks,inertias_mini,marker='x')# want to use elbow method to find best k
    ax=fig.add_subplot(2,1,1)
    ax=sns.barplot(data=sorted_weight_data, 
                   x="scaled_weights", 
                   y="features", 
                   palette="Blues_d")
    ax.set_title("Cluster centroid")

    ax=fig.add_subplot(2,1,2)
    ax=sns.barplot(data=sorted_weight_data, 
                   x="original_weights", 
                   y="features", 
                   palette="Blues_d")
    ax.set_title("Cluster centroid")
    plt.show()
display_component(kmeans.cluster_centers_[centroid_id], max_azdias, min_azdias)

In [ ]:
def print_component(centroid, max_azdias, min_azdias, n_weights = 25):
    comp = pd.DataFrame(list(zip(centroid, list(azdias.columns))), columns=['scaled_weights', 'features'])
    comp['abs_weights'] = comp['scaled_weights'].apply(lambda x: np.abs(x))
    comp['original_weights'] = comp.apply(lambda row: (row.scaled_weights*(max_azdias[row.features] - min_azdias[row.features]) + min_azdias[row.features]), axis = 1) 
    sorted_weight_data = comp.sort_values('abs_weights', ascending=False).head(n_weights)
    #print(sorted_weight_data)

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('Udacity_MAILOUT_052018_TRAIN.csv', sep=';')


In [ ]:
fill_unknown_values(mailout_train)

# handle string
convert_str_to_int(mailout_train, "CAMEO_DEUG_2015")
convert_str_to_int(mailout_train, "CAMEO_INTL_2015")

convert_biclass_to_int(mailout_train, "OST_WEST_KZ")
try:
    mailout_train = convert_categories_to_onehot(mailout_train, "CAMEO_DEU_2015")
    mailout_train = convert_categories_to_onehot(mailout_train, "D19_LETZTER_KAUF_BRANCHE")
except Exception as e:
    print(e)

# handle time stamps
mailout_train['EINGEFUEGT_AM'] = pd.to_datetime(mailout_train['EINGEFUEGT_AM']).sub(pd.Timestamp('1900-01-01')).dt.days

for column_name in mailout_train.columns:
    
    keys = mailout_train.groupby(column_name).groups.keys() 
    if set(keys) <= set(range(50)):
        continue
    print(column_name)

    print(keys)

In [ ]:
for c in mailout_train.columns:
    mailout_train[c].fillna(mailout_train[c].mean(), inplace=True)
print([col for col in mailout_train.columns if np.sum(mailout_train[col].isna()) > 0])

mailout_train.to_pickle("mailout_train.pkl")

In [ ]:
mailout_train = pd.read_pickle("mailout_train.pkl")

max_mailout_train = mailout_train.max()
min_mailout_train = mailout_train.min()
normalized_mailout_train = (mailout_train - min_mailout_train)/(max_mailout_train - min_mailout_train)
normalized_mailout_train.describe()

In [ ]:
azdias_rotated = pca.fit_transform(normalized_mailout_train)
explained_variance_by_k = pca.explained_variance_ratio_.cumsum()
plt.plot(range(1,len(explained_variance_by_k)+1),explained_variance_by_k,marker="*")
plt.xlabel("Num. of Components")
plt.ylabel("Cumulative Power Ratio")

In [ ]:
from sklearn.cross_validation import  train_test_split
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
from xgboost import XGBClassifier

seed = 999
X = normalized_mailout_train.loc[:,normalized_mailout_train.columns != 'RESPONSE']
y = normalized_mailout_train["RESPONSE"]

# split into training dataset and test dataset
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=seed)
train_matrix = xgb.DMatrix(Xtrain, ytrain)
test_matrix = xgb.DMatrix(Xtest)

In [ ]:
params = {}
#params['silent'] = 1
params['objective'] = 'binary:logistic'  # output probabilities
params['eval_metric'] = 'auc'
params["num_rounds"] = 300
params["early_stopping_rounds"] = 30
# params['min_child_weight'] = 2
params['max_depth'] = 6
params['eta'] = 0.1
params["subsample"] = 0.8
params["colsample_bytree"] = 0.8

cv_results = xgb.cv(params, train_matrix,
                    num_boost_round = params["num_rounds"],
                    nfold = params.get('nfold',5),
                    metrics = params['eval_metric'],
                    early_stopping_rounds = params["early_stopping_rounds"],
                    verbose_eval = True,
                    seed = seed)

In [ ]:
cv_results

In [ ]:
n_best_trees = cv_results.shape[0]
n_best_trees

In [ ]:
watchlist = [(train_matrix, 'train')]
gbt = xgb.train(params, train_matrix, n_best_trees,watchlist)

In [ ]:
from sklearn.metrics import roc_curve, auc
def plot_validation_roc():
    Xtrain_only,Xvalid,ytrain_only,yvalid = train_test_split(Xtrain,ytrain,test_size=0.3,random_state=seed)
    print(Xvalid.shape)
    #onlytrain_matrix = xgb.DMatrix(Xtrain_only,ytrain_only)
    valid_matrix = xgb.DMatrix(Xvalid)

    temp_gbt = gbt#xgb.train(params, onlytrain_matrix, n_best_trees,[(onlytrain_matrix,'train_only'),(valid_matrix,'validate')])
    yvalid_proba_pred = gbt.predict(valid_matrix,ntree_limit=n_best_trees)
    print(len(yvalid_proba_pred))
    fpr,tpr,thresholds = roc_curve(yvalid,yvalid_proba_pred)
    print("auc = {}".format(auc(fpr,tpr)))
    return pd.DataFrame({'FPR':fpr,'TPR':tpr,'Threshold':thresholds})

roc = plot_validation_roc()
plt.figure(figsize=(10,5))
plt.plot(roc.FPR,roc.TPR,marker='h')
plt.xlabel("FPR")
plt.ylabel("TPR")


## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('Udacity_MAILOUT_052018_TEST.csv', sep=';')

In [ ]:
fill_unknown_values(mailout_test)

# handle string
convert_str_to_int(mailout_test, "CAMEO_DEUG_2015")
convert_str_to_int(mailout_test, "CAMEO_INTL_2015")

convert_biclass_to_int(mailout_test, "OST_WEST_KZ")

mailout_test = convert_categories_to_onehot(mailout_test, "CAMEO_DEU_2015")
mailout_test = convert_categories_to_onehot(mailout_test, "D19_LETZTER_KAUF_BRANCHE")


# handle time stamps
mailout_test['EINGEFUEGT_AM'] = pd.to_datetime(mailout_test['EINGEFUEGT_AM']).sub(pd.Timestamp('1900-01-01')).dt.days

for c in mailout_test.columns:
    mailout_test[c].fillna(mailout_test[c].mean(), inplace=True)
print([col for col in mailout_test.columns if np.sum(mailout_test[col].isna()) > 0])

max_mailout_test = mailout_test.max()
min_mailout_test = mailout_test.min()
normalized_mailout_test = (mailout_test - min_mailout_test)/(max_mailout_test - min_mailout_test)
normalized_mailout_test.describe()

In [ ]:

print(normalized_mailout_test.shape)
test_matrix = xgb.DMatrix(normalized_mailout_test)
pred = gbt.predict(test_matrix, ntree_limit=n_best_trees)
print(len(pred))
df_pred_test = pd.DataFrame({"RESPONSE":pred})


In [ ]:
mailout_test_save = pd.read_csv('Udacity_MAILOUT_052018_TEST.csv', sep=';')
df = pd.DataFrame({"LNR":mailout_test_save["LNR"], "RESPONSE":pred})
print(df.columns)
df.to_csv("test_pred.csv",index=False)